In [61]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from jdatetime import datetime as jdatetime
from datetime import timedelta


> **Let`s import the Dataset and see some information about it.**




In [62]:
data=pd.read_excel('/content/Survey_Report.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 39 columns):
 #   Column                                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                                --------------  ----- 
 0   پاسخنامه                                                                                                                                              91 non-null     object
 1   شناسه پاسخ دهنده                                                                                                                                      91 non-null     object
 2   لطفا آدرس ایمیل خود را وارد کنید.                                                                                                                     89 non-null     object
 3   لط



> **There are some null values in the Email and Medical Registration columns. It is essential to clearly identify the source of any submitted opinions, as there is a risk that they may be generated by automated systems, rendering them unreliable. Therefore, these entries should be excluded.**

In [63]:
data.dropna(subset=[data.columns[2], data.columns[3]], inplace=True)



> **Before encoding sensitive personal data such as email addresses and Medical Registration numbers in the dataset, it is essential to:**
1. **validate the email addresses.**
2. **ensure that each individual, identified by a unique Email address or Medical Registration number, has not submitted duplicate enteries.**
3. **Remove any entries containing confidential information.**





> **1. validate the email addresses.**



In [64]:
Emails = pd.Series(data.iloc[:,2])

def is_valid_email(email):
    return isinstance(email, str) and re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', email)

invalid_emails = [email for email in Emails if not is_valid_email(email)]

if invalid_emails:
    for email in invalid_emails:
        print(f'{email} has an invalid format or is not a string')
else:
    print('All registered Emails have a valid format')


All registered Emails have a valid format




> **2. Ensure no duplicate entries are submitted.**



In [65]:
is_duplicated = data.duplicated(subset=data.columns[2], keep=False) | data.duplicated(subset=data.columns[3], keep=False)
duplicated_indices = data[is_duplicated].index

if not duplicated_indices.empty:
    print("Duplicated indices:")
    print(duplicated_indices.tolist())
else:
    print("Nothing is duplicated")

Nothing is duplicated




> **3. Drop confidential information.**



In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 2 to 86
Data columns (total 39 columns):
 #   Column                                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                                --------------  ----- 
 0   پاسخنامه                                                                                                                                              85 non-null     object
 1   شناسه پاسخ دهنده                                                                                                                                      85 non-null     object
 2   لطفا آدرس ایمیل خود را وارد کنید.                                                                                                                     85 non-null     object
 3   لطفا کد

In [67]:
data.drop(data.columns[[0,1,35,36]], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 2 to 86
Data columns (total 35 columns):
 #   Column                                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                                --------------  ----- 
 0   لطفا آدرس ایمیل خود را وارد کنید.                                                                                                                     85 non-null     object
 1   لطفا کد نظام پزشکی یا شماره دانشجویی خود را وارد کنید.                                                                                                85 non-null     object
 2   تا کنون بیشتر در چه حوزه‌ه‍ایی به پژوهش پرداخته اید؟                                                                                                  45 non-null     object
 3   Unnamed



> **Let`s Encode Email addresses and Medical Registration numbers 😎**



In [68]:
def mask_email(email):
    if isinstance(email, str):
        parts = email.split('@')
        local = parts[0]
        domain = parts[1]
        masked_local = local[0] + '*' * (len(local) - 2) + local[-1] if len(local) > 2 else local
        masked_domain = domain[0] + '*' * (len(domain) - 2) + domain[-1] if len(domain) > 2 else domain
        return masked_local + '@' + masked_domain
    return email

data.iloc[:,0] = data.iloc[:,0].apply(mask_email)

data.iloc[:,0]

2        A**************b@y*******m
3     f*****************8@g*******m
4            r**********j@g*******m
5            a**********0@y*******m
6           s***********m@g*******m
                  ...              
82              R*******9@g*******m
83       F**************3@g*******m
84        m*************n@g*******m
85            a*********6@g*******m
86    s*****************1@g*******m
Name: لطفا آدرس ایمیل خود را وارد کنید., Length: 85, dtype: object

In [69]:
def mask_Medical_Registration_num(Medical_Registration_num):
    if isinstance(Medical_Registration_num, str):
        length = len(Medical_Registration_num)
        if length <= 2:
            return '*' * length
        else:
            masked_number = Medical_Registration_num[0] + '*' * (length - 2) + Medical_Registration_num[-1]
            return masked_number
    return Medical_Registration_num

data.iloc[:,1] = data.iloc[:,1].apply(mask_Medical_Registration_num)

data.iloc[:,1]

2          ت*****۰
3             9**9
4           ت****۳
5            8***0
6       9********2
          ...     
82           1***1
83            ۹**۸
84            7**0
85           9***T
86    1**********5
Name: لطفا کد نظام پزشکی یا شماره دانشجویی خود را وارد کنید., Length: 85, dtype: object



> **Now you can see the whole data 🍓**



In [70]:
data.head()

,لطفا آدرس ایمیل خود را وارد کنید.,لطفا کد نظام پزشکی یا شماره دانشجویی خود را وارد کنید.,تا کنون بیشتر در چه حوزه‌ه‍ایی به پژوهش پرداخته اید؟,Unnamed: 5,Unnamed: 6,Unnamed: 7,به کدام دسته از کارهای پژوهشی بیشتر پرداخته اید؟,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,استفاده از کدامیک خطای بیشتر و دقت کمتری دارد؟,Unnamed: 28,Unnamed: 29,Unnamed: 30,اگر قرار باشد آن را اصلاح کنید چه تغییراتی در آنها ایجاد می‌­کنید؟,چنانچه در راستای حل مشکلاتی که اشاره فرمودید خدماتی به شما ارائه گردد، آیا تمایل خواهید داشت از این خدمات استفاده کنید؟,در صورت اصلاح مشکلات موجود در این موارد، به طور میانگین چند درصد از مبلغ هزینه شده برای انجام پژوهش کاهش می‌یابد؟,فکر می‌کنید چه هزینه‌ای برای نرم افزاری که بتواند مشکلات را حل کند مناسب است؟,تاریخ شروع,تاریخ اتمام
2,A**************b@y*******m,ت*****۰,تغذیه بالینی,NaN,NaN,NaN,مورد شاهدی,NaN,NaN,مقطعی,...,استفاده از پرسشنامه جهت بررسی وضعیت دریافت غذایی,NaN,NaN,NaN,نظری ندارم,بله,اطلاعی ندارم,زیر ۱ میلیون یا اشتراک سالانه زیر پانصد هزار تومن,1403/03/27-21:26:56,1403/03/27-21:29:34
3,f*****************8@g*******m,9**9,NaN,تغذیه جامعه,NaN,NaN,مورد شاهدی,NaN,NaN,مقطعی,...,NaN,NaN,وارد کردن اطلاعات دریافت شده از پرسشنامه به نر...,NaN,NaN,بله,NaN,NaN,1403/03/27-10:51:50,1403/03/27-10:55:00
4,r**********j@g*******m,ت****۳,تغذیه بالینی,NaN,تغذیه سلولی و مولکولی,NaN,NaN,RCT,مروری,مقطعی,...,NaN,شکاف بین اطلاعات نوشته شده توسط شرکت کننده و ب...,NaN,NaN,گذاشتن زمان بیشتر برای آگاه سازی و توانمند کرد...,بله,۲۰ تا ۳۰ درصد,NaN,1403/03/27-10:15:18,1403/03/27-10:24:45
5,a**********0@y*******m,8***0,تغذیه بالینی,NaN,NaN,NaN,NaN,RCT,NaN,NaN,...,NaN,NaN,وارد کردن اطلاعات دریافت شده از پرسشنامه به نر...,NaN,online connection,بله,NaN,NaN,1403/03/26-13:30:21,1403/03/26-13:32:51
6,s***********m@g*******m,9********2,NaN,تغذیه جامعه,NaN,NaN,مورد شاهدی,NaN,NaN,مقطعی,...,NaN,NaN,وارد کردن اطلاعات دریافت شده از پرسشنامه به نر...,NaN,مردم حوصله پاسخگویی دقیق ندارند و نمی شود این ...,بله,نمیدانم. بر سرعت کار اثر دارد,مشکل اصلی زمان بر بودن و بومی نبودن نرم افزاره...,1403/03/26-13:11:08,1403/03/26-13:18:07




> **As indicated earlier, this dataset is entirely in Persian, presenting numerous tasks that require attention. 🤦‍♂️🫠**





> **In this survey, the questions are structured such that the omission of any single response results in the invalidation of all other provided information. Consequently, the data from these respondents should be deleted.**



In [71]:
dropped_indexes = []

columns_to_check_groups = [
    data.columns[2:6],
    data.columns[6:12],
    data.columns[12:16],
    data.columns[16:20],
    data.columns[21:25],
    data.columns[25:29]
]

for columns_to_check in columns_to_check_groups:
    indexes_to_drop = data[data[columns_to_check].isna().all(axis=1)].index
    dropped_indexes.extend(indexes_to_drop)
    data = data.drop(indexes_to_drop)

dropped_indexes = sorted(set(dropped_indexes))
dropped_indexes

[31, 32, 34, 36, 43, 47, 58, 61, 81, 85]

In [72]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75 entries, 2 to 86
Data columns (total 35 columns):
 #   Column                                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                                --------------  ----- 
 0   لطفا آدرس ایمیل خود را وارد کنید.                                                                                                                     75 non-null     object
 1   لطفا کد نظام پزشکی یا شماره دانشجویی خود را وارد کنید.                                                                                                75 non-null     object
 2   تا کنون بیشتر در چه حوزه‌ه‍ایی به پژوهش پرداخته اید؟                                                                                                  43 non-null     object
 3   Unnamed



> **Another crucial metric for validating the data is the duration spent by respondents on each answer. Let's delve into this aspect to gain deeper insights.**



In [73]:
def jalali_to_seconds(jalali_date_str):
    dt = jdatetime.strptime(jalali_date_str, '%Y/%m/%d-%H:%M:%S')
    return dt.hour * 3600 + dt.minute * 60 + dt.second

for i in range(len(data)):
    start_time = data.iloc[i, 33]
    end_time = data.iloc[i, 34]
    dt1 = jdatetime.strptime(start_time, '%Y/%m/%d-%H:%M:%S')
    dt2 = jdatetime.strptime(end_time, '%Y/%m/%d-%H:%M:%S')

    if dt1.day < dt2.day:
        duration_seconds = (86400 - jalali_to_seconds(start_time)) + jalali_to_seconds(end_time)
    else:
        duration_seconds = jalali_to_seconds(end_time) - jalali_to_seconds(start_time)

    data.at[i, 'Duration_Seconds'] = duration_seconds



> **To determine a reasonable minimum time threshold for your survey responses, it's important to balance between filtering out hasty or careless responses and retaining legitimate ones. Given your data, let's analyze the distribution and provide a more informed recommendation.**



In [74]:
data['Duration_Seconds'].describe()

count        75.000000
mean       2309.053333
std       11968.733941
min         140.000000
25%         236.500000
50%         344.000000
75%         589.000000
max      102169.000000
Name: Duration_Seconds, dtype: float64


**Suggested Threshold: 200 seconds**

**1. Above Minimum: Setting the threshold at 200 seconds
is above the minimum recorded time of 140 seconds, ensuring a buffer for genuine but fast responses.**

**2. Closer to Q1: It is slightly below the 25th percentile (236.5 seconds), capturing a reasonable lower bound while still filtering out potentially hasty responses.**

**3. Retention of Genuine Responses: This threshold helps retain responses that are likely genuine but completed quickly by fast readers or knowledgeable participants.**


In [79]:
data = data[data['Duration_Seconds'] >= 200]
data.drop(columns=data.columns[33:], inplace=True)

In [81]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 2 to 61
Data columns (total 33 columns):
 #   Column                                                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                                                --------------  ----- 
 0   لطفا آدرس ایمیل خود را وارد کنید.                                                                                                                     52 non-null     object
 1   لطفا کد نظام پزشکی یا شماره دانشجویی خود را وارد کنید.                                                                                                52 non-null     object
 2   تا کنون بیشتر در چه حوزه‌ه‍ایی به پژوهش پرداخته اید؟                                                                                                  29 non-null     object
 3   Unnamed

In [82]:
data.head()

,لطفا آدرس ایمیل خود را وارد کنید.,لطفا کد نظام پزشکی یا شماره دانشجویی خود را وارد کنید.,تا کنون بیشتر در چه حوزه‌ه‍ایی به پژوهش پرداخته اید؟,Unnamed: 5,Unnamed: 6,Unnamed: 7,به کدام دسته از کارهای پژوهشی بیشتر پرداخته اید؟,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 25,Unnamed: 26,استفاده از کدامیک خطای بیشتر و دقت کمتری دارد؟,Unnamed: 28,Unnamed: 29,Unnamed: 30,اگر قرار باشد آن را اصلاح کنید چه تغییراتی در آنها ایجاد می‌­کنید؟,چنانچه در راستای حل مشکلاتی که اشاره فرمودید خدماتی به شما ارائه گردد، آیا تمایل خواهید داشت از این خدمات استفاده کنید؟,در صورت اصلاح مشکلات موجود در این موارد، به طور میانگین چند درصد از مبلغ هزینه شده برای انجام پژوهش کاهش می‌یابد؟,فکر می‌کنید چه هزینه‌ای برای نرم افزاری که بتواند مشکلات را حل کند مناسب است؟
2,A**************b@y*******m,ت*****۰,تغذیه بالینی,NaN,NaN,NaN,مورد شاهدی,NaN,NaN,مقطعی,...,NaN,NaN,استفاده از پرسشنامه جهت بررسی وضعیت دریافت غذایی,NaN,NaN,NaN,نظری ندارم,بله,اطلاعی ندارم,زیر ۱ میلیون یا اشتراک سالانه زیر پانصد هزار تومن
4,r**********j@g*******m,ت****۳,تغذیه بالینی,NaN,تغذیه سلولی و مولکولی,NaN,NaN,RCT,مروری,مقطعی,...,NaN,NaN,NaN,شکاف بین اطلاعات نوشته شده توسط شرکت کننده و ب...,NaN,NaN,گذاشتن زمان بیشتر برای آگاه سازی و توانمند کرد...,بله,۲۰ تا ۳۰ درصد,NaN
5,a**********0@y*******m,8***0,تغذیه بالینی,NaN,NaN,NaN,NaN,RCT,NaN,NaN,...,NaN,NaN,NaN,NaN,وارد کردن اطلاعات دریافت شده از پرسشنامه به نر...,NaN,online connection,بله,NaN,NaN
6,s***********m@g*******m,9********2,NaN,تغذیه جامعه,NaN,NaN,مورد شاهدی,NaN,NaN,مقطعی,...,NaN,NaN,NaN,NaN,وارد کردن اطلاعات دریافت شده از پرسشنامه به نر...,NaN,مردم حوصله پاسخگویی دقیق ندارند و نمی شود این ...,بله,نمیدانم. بر سرعت کار اثر دارد,مشکل اصلی زمان بر بودن و بومی نبودن نرم افزاره...
7,m*********2@y*******m,\n****ت,NaN,تغذیه جامعه,NaN,NaN,NaN,NaN,NaN,مقطعی,...,NaN,NaN,NaN,شکاف بین اطلاعات نوشته شده توسط شرکت کننده و ب...,وارد کردن اطلاعات دریافت شده از پرسشنامه به نر...,NaN,پرسشگری و آنالیز به شکل کاملا مکانیزه,بله,50,اطلاع ندارم





---


> ***After rigorous filtering steps, we have refined the dataset from 93 entries to 59, ensuring that only data conducive to informed decision-making remains. This dataset is now primed for advanced and specialized analysis, paving the way for deeper insights and strategic next steps.***



---

